# 🚀 JetX Model Eğitimi - Google Colab (v5.0 + Consensus)

**Bu notebook ile tüm JetX tahmin modellerini Multi-Scale Window Ensemble sistemi ile eğitebilir ve Consensus modelini test edebilirsiniz.**

---

## 🆕 YENİ: Consensus Model Sistemi (v5.0)

### 🎯 Consensus Nedir?

**İki farklı algoritma (NN ve CatBoost) aynı tahmin yaptığında bahis yapma sistemi:**
- ✅ **Her iki model de 1.5 üstü derse** → OYNA
- ❌ **Farklı tahmin yaparlarsa** → OYNAMA

**İki Sanal Kasa Stratejisi:**
- **Kasa 1**: 1.5x eşikte çık
- **Kasa 2**: İki modelin regression tahminlerinin ortalamasının %70'inde çık

---

## 🎯 Eğitim Sırası:

### 1️⃣ Progressive NN - Multi-Scale Ensemble
- ⏱️ **Süre**: ~10-12 saat (GPU ile)
- 🔥 **5 Model**: Her pencere için LSTM-based model

### 2️⃣ CatBoost - Multi-Scale Ensemble
- ⏱️ **Süre**: ~40-60 dakika
- ⚡ **10 Model**: 5 pencere × 2 (Regressor + Classifier)

### 3️⃣ Consensus Değerlendirme
- ⏱️ **Süre**: ~5-10 dakika
- 🎯 **İki Modeli Birleştirir**: NN + CatBoost

---

## 📌 Kullanım:

1. **Runtime → Change runtime type → GPU (T4 önerilir)**
2. **Adım 1'i çalıştırın** (Hazırlık)
3. **Adım 2A'yı çalıştırın** (Progressive NN eğitimi)
4. **Adım 2B'yi çalıştırın** (CatBoost eğitimi)
5. **Adım 2C'yi çalıştırın** (Consensus değerlendirme)
6. **Model indirme adımlarını çalıştırın**
7. **ZIP dosyalarını lokal projenize kopyalayın**

# 📦 Adım 1: Hazırlık (Tüm Seçenekler İçin Gerekli)

In [ ]:
import subprocess
import sys
import os

print("="*80)
print("📦 HAZIRLIK - Multi-Scale + Consensus Sistem v5.0")
print("="*80)

# Kütüphaneleri yükle
print("\n📦 Kütüphaneler yükleniyor...")
!pip install -q tensorflow scikit-learn catboost pandas numpy scipy joblib matplotlib seaborn tqdm PyWavelets nolds

# Proje yükle
if os.path.exists('jetxpredictor'):
    !rm -rf jetxpredictor

print("\n📥 Proje GitHub'dan indiriliyor...")
!git clone https://github.com/onndd/jetxpredictor.git
%cd jetxpredictor

print("\n✅ Hazırlık tamamlandı!")
print("\n➡️ Eğitim sırası:")
print("   1. Adım 2A: Progressive NN Multi-Scale (~10-12 saat)")
print("   2. Adım 2B: CatBoost Multi-Scale (~40-60 dakika)")
print("   3. Adım 2C: Consensus Değerlendirme (~5-10 dakika)")

---

# 🧠 ADIM 2A: Progressive NN - Multi-Scale Ensemble

**⏱️ Tahmini Süre: 10-12 saat (GPU ile)**

**🆕 Multi-Scale Özellikleri:**
- 5 farklı pencere boyutu için ayrı LSTM modelleri
- Küçük pencereler (20, 50): Basit LSTM
- Orta pencereler (100, 250): 2-layer LSTM
- Büyük pencereler (500): 3-layer LSTM + Attention
- Ensemble prediction: Tüm modellerin ortalaması
- **Weighted Model Selection**: Her epoch'ta karlılık bazlı model seçimi

In [ ]:
print("🧠 PROGRESSIVE NN - Multi-Scale Window Ensemble Başlatılıyor...")
print("⏱️ Tahmini süre: 10-12 saat (GPU ile)")
print("🆕 5 Model: Her pencere boyutu için ayrı model")
print("📊 Window boyutları: [500, 250, 100, 50, 20]")
print("✨ Weighted Selection: 50% Below + 40% Above + 10% ROI")
print("="*70)

!python notebooks/jetx_PROGRESSIVE_TRAINING_MULTISCALE.py

print("\n" + "="*70)
print("✅ Progressive NN Multi-Scale eğitimi tamamlandı!")
print("✅ 5 model başarıyla eğitildi ve ensemble edildi")
print("\n➡️ Şimdi 'Adım 3A: Model İndirme' hücresini çalıştırın")
print("➡️ Sonra 'Adım 2B: CatBoost Eğitimi' ile devam edin")

### Adım 3A: Progressive NN Multi-Scale Modelini İndir

In [ ]:
import os

print("💾 Progressive NN Multi-Scale dosyaları indiriliyor...\n")
print("="*80)

zip_file = 'jetx_models_progressive_multiscale_v3.0.zip'

if os.path.exists(zip_file):
    print(f"✅ {zip_file} bulundu!\n")
    
    try:
        from google.colab import files
        print(f"📥 Otomatik indirme başlatılıyor...")
        files.download(zip_file)
        print(f"✅ {zip_file} başarıyla indirildi!\n")
    except Exception as e:
        print(f"⚠️ Otomatik indirme başarısız: {e}\n")
        print("📁 MANUEL İNDİRME: Sol panelden Files → jetxpredictor → " + zip_file + " → Sağ tık → Download")
    
    print("\n📦 ZIP İÇERİĞİ (5 Window × 1 Model = 5 Model):")
    print("   • model_window_500.h5")
    print("   • model_window_250.h5")
    print("   • model_window_100.h5")
    print("   • model_window_50.h5")
    print("   • model_window_20.h5")
    print("   • scaler_window_*.pkl (Her pencere için)")
    print("   • model_info.json")
    
    print("\n🆕 MULTI-SCALE ÖZELLİKLERİ:")
    print("   ✅ 5 farklı zaman ölçeğinde öğrenme")
    print("   ✅ Ensemble prediction sistemi")
    print("   ✅ Weighted model selection")
else:
    print(f"⚠️ {zip_file} bulunamadı!")

print("\n🎉 Kullanıma hazır!")

---

# ⚡ ADIM 2B: CatBoost - Multi-Scale Ensemble

**⏱️ Tahmini Süre: 40-60 dakika**

**🆕 Multi-Scale Özellikleri:**
- 5 pencere × 2 model (Regressor + Classifier) = 10 model
- Küçük pencereler: Agresif weights (25x)
- Büyük pencereler: Dengeli weights (10x)
- Ensemble prediction: Tüm modellerin ortalaması

In [ ]:
print("⚡ CATBOOST - Multi-Scale Window Ensemble Başlatılıyor...")
print("⏱️ Tahmini süre: 40-60 dakika")
print("🆕 10 Model: 5 pencere × 2 model (Reg + Cls)")
print("📊 Window boyutları: [500, 250, 100, 50, 20]")
print("="*70)

!python notebooks/jetx_CATBOOST_TRAINING_MULTISCALE.py

print("\n" + "="*70)
print("✅ CatBoost Multi-Scale eğitimi tamamlandı!")
print("✅ 10 model başarıyla eğitildi ve ensemble edildi")
print("\n➡️ Şimdi 'Adım 3B: Model İndirme' hücresini çalıştırın")
print("➡️ Sonra 'Adım 2C: Consensus Değerlendirme' ile devam edin")

### Adım 3B: CatBoost Multi-Scale Modelini İndir

In [ ]:
import os

print("💾 CatBoost Multi-Scale dosyaları indiriliyor...\n")
print("="*80)

zip_file = 'jetx_models_catboost_multiscale_v3.0.zip'

if os.path.exists(zip_file):
    print(f"✅ {zip_file} bulundu!\n")
    
    try:
        from google.colab import files
        print(f"📥 Otomatik indirme başlatılıyor...")
        files.download(zip_file)
        print(f"✅ {zip_file} başarıyla indirildi!\n")
    except Exception as e:
        print(f"⚠️ Otomatik indirme başarısız: {e}\n")
        print("📁 MANUEL İNDİRME: Sol panelden Files → jetxpredictor → " + zip_file + " → Sağ tık → Download")
    
    print("\n📦 ZIP İÇERİĞİ (5 Window × 2 Model = 10 Model):")
    print("   • regressor_window_*.cbm (5 adet)")
    print("   • classifier_window_*.cbm (5 adet)")
    print("   • scaler_window_*.pkl (Her pencere için)")
    print("   • model_info.json")
else:
    print(f"⚠️ {zip_file} bulunamadı!")

print("\n🎉 Kullanıma hazır!")

---

# 🎯 ADIM 2C: Consensus Model Değerlendirme (YENİ!)

**⏱️ Tahmini Süre: 5-10 dakika**

**🆕 Consensus Özellikleri:**
- NN ve CatBoost modellerini birleştirir
- Sadece her iki model de 1.5 üstü derse bahis yapar
- İki farklı kasa stratejisini test eder:
  - **Kasa 1**: 1.5x eşikte çık
  - **Kasa 2**: Ortalama tahmin × %70'te çık

⚠️ **NOT**: Bu adımı çalıştırmadan önce hem NN hem de CatBoost modellerinin eğitilmiş olması gerekir!

In [ ]:
print("🎯 CONSENSUS MODEL DEĞERLENDİRME Başlatılıyor...")
print("⏱️ Tahmini süre: 5-10 dakika")
print("🆕 İki Modeli Birleştirir: NN + CatBoost")
print("📊 İki Kasa Stratejisi: 1.5x ve %70 çıkış")
print("="*70)

!python notebooks/CONSENSUS_EVALUATION.py

print("\n" + "="*70)
print("✅ Consensus değerlendirme tamamlandı!")
print("✅ Sonuçlar results/consensus_evaluation.json dosyasına kaydedildi")
print("\n➡️ Sonuçları görmek için aşağıdaki hücreyi çalıştırın")

### Consensus Sonuçlarını Görüntüle

In [ ]:
import json
import os

results_file = 'results/consensus_evaluation.json'

if os.path.exists(results_file):
    with open(results_file, 'r') as f:
        results = json.load(f)
    
    print("="*80)
    print("🎯 CONSENSUS MODEL SONUÇLARI")
    print("="*80)
    
    print(f"\n📊 CONSENSUS İSTATİSTİKLERİ:")
    print(f"  Toplam Tahmin: {results['consensus_stats']['total_predictions']:,}")
    print(f"  Consensus Oranı: {results['consensus_stats']['consensus_rate']*100:.1f}%")
    print(f"  Consensus Doğruluğu: {results['consensus_stats']['consensus_accuracy']:.1f}%")
    
    print(f"\n💰 KASA 1 (1.5x EŞİK):")
    kasa1 = results['kasa_1_15x']
    print(f"  ROI: {kasa1['roi']:+.2f}%")
    print(f"  Win Rate: {kasa1['win_rate']:.1f}%")
    print(f"  Toplam Bahis: {kasa1['total_bets']:,}")
    print(f"  Kar/Zarar: {kasa1['profit']:+,.2f} TL")
    
    print(f"\n💰 KASA 2 (%70 ÇIKIŞ):")
    kasa2 = results['kasa_2_70pct']
    print(f"  ROI: {kasa2['roi']:+.2f}%")
    print(f"  Win Rate: {kasa2['win_rate']:.1f}%")
    print(f"  Toplam Bahis: {kasa2['total_bets']:,}")
    print(f"  Kar/Zarar: {kasa2['profit']:+,.2f} TL")
    
    print(f"\n📈 KARŞILAŞTIRMA:")
    roi_diff = kasa2['roi'] - kasa1['roi']
    print(f"  ROI Farkı: {roi_diff:+.2f}%")
    if kasa2['roi'] > kasa1['roi']:
        print(f"  ✅ Kasa 2 (%70 çıkış) daha iyi!")
    else:
        print(f"  ✅ Kasa 1 (1.5x eşik) daha iyi!")
    
    print("\n" + "="*80)
else:
    print(f"⚠️ Sonuç dosyası bulunamadı: {results_file}")
    print("Önce Consensus değerlendirmesini çalıştırın!")

---

# 📋 Önemli Notlar

## Consensus Sistemi Avantajları:

**Yüksek Güven:**
- İki farklı algoritma hemfikir olduğunda tahmin daha güvenilir
- Risk azaltma: Sadece consensus varsa oynanır
- False positive filtreleme: Bir model hata yaparsa diğeri dengeleyebilir

**İki Kasa Stratejisi:**
- **Kasa 1 (1.5x)**: Basit ve muhafazakâr
- **Kasa 2 (%70)**: Dinamik çıkış, daha yüksek win rate beklenir

## Modelleri Kullanma:

1. **ZIP dosyalarını açın**
2. **Dosyaları `models/` klasörüne kopyalayın**
3. **Streamlit uygulamasını başlatın**: `python -m streamlit run app.py`

## Uyarılar:

- 🚨 Bu modeller %100 doğru değildir
- 💰 Yine de para kaybedebilirsiniz
- 🎯 Consensus sistemi daha muhafazakâr ve güvenlidir
- ✨ İki model hemfikir olduğunda güven seviyesi daha yüksektir

**Sorumlu oynayın! 🎲**